In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import Packages.ClusteringHelper as ch
import numpy as np
from tqdm import tqdm
from collections import Counter
import string
import re
from nltk.corpus import stopwords

In [4]:
text, data = ch.read_aida_yago_conll(
    "D:\\Sgmon\\Documents\\Magistrale\\TESI\\ClusteringAndLinking\\aida-yago2-dataset\\AIDA-YAGO2-dataset.tsv")
save = False
if save:
    text_file = open('text.txt', 'w')
    text_file.write(text)
    text_file.close()
data

,documents,tokens,indexes,word_indexes,mentions,entities,wikidatas,numeric_codes,alpha_codes
0,0,EU,"(0, 2)",0,,,,,
1,0,rejects,"(3, 10)",1,,,,,
2,0,German,"(11, 17)",2,German,Germany,http://en.wikipedia.org/wiki/Germany,11867,/m/0345h
3,0,call,"(18, 22)",3,,,,,
4,0,to,"(23, 25)",4,,,,,
...,...,...,...,...,...,...,...,...,...
285584,1392,younger,"(1342, 1349)",265,,,,,
285585,1392,brother,"(1350, 1357)",266,,,,,
285586,1392,",","(1358, 1359)",267,,,,,
285587,1392,Bobby,"(1360, 1365)",268,Bobby,Bobby_Charlton,http://en.wikipedia.org/wiki/Bobby_Charlton,4224,/m/01c8x


In [5]:
data = ch.filter_data(data, 3)
data

,documents,tokens,indexes,word_indexes,mentions,entities,wikidatas,numeric_codes,alpha_codes
0,0,EU,"(0, 2)",0,,,,,
1,0,rejects,"(3, 10)",1,,,,,
2,0,German,"(11, 17)",2,German,Germany,http://en.wikipedia.org/wiki/Germany,11867,/m/0345h
3,0,call,"(18, 22)",3,,,,,
4,0,to,"(23, 25)",4,,,,,
...,...,...,...,...,...,...,...,...,...
285584,1392,younger,"(1342, 1349)",265,,,,,
285585,1392,brother,"(1350, 1357)",266,,,,,
285586,1392,",","(1358, 1359)",267,,,,,
285587,1392,Bobby,"(1360, 1365)",268,,,http://en.wikipedia.org/wiki/Bobby_Charlton,4224,/m/01c8x


In [6]:
# Gold standard
ents_data = data[data['entities'] != '']
entities = ents_data['entities'].values
mentions = ents_data['tokens'].values
docs = ents_data['documents'].values
word_indexes = ents_data['word_indexes'].values
unique_entities = np.unique(entities)
gold_standard = Counter(entities)
gold_standard = dict(sorted(gold_standard.items(), key=lambda item: item[1], reverse=True))

#### Gensim word2vec

In [7]:
lowercase = True
remove_stopwords = True
remove_numbers = True
remove_symbols = True

text_preprocessed = text
stop_words = set(stopwords.words("english"))

if remove_symbols:
    text_preprocessed = [x.translate(str.maketrans('', '', string.punctuation)) for x in text_preprocessed]

# rimuovo numeri e punteggiatura
if lowercase:
    text_splitted_training = [x.lower().split() for x in text_preprocessed]
    text_splitted = [x.lower().split() for x in text]
else:
    text_splitted_training = [x.split() for x in text_preprocessed]
    text_splitted = [x.split() for x in text]

if remove_numbers:
    text_splitted_training = [[y for y in x if not y.isnumeric()] for x in text_splitted_training]

if remove_stopwords:
    text_splitted_training = [[y for y in x if y.lower() not in stop_words] for x in text_splitted_training]

Model selection

In [8]:
import os
from gensim.models import KeyedVectors

model_path = "D:\Sgmon\Documents\Magistrale\TESI\gensim-data"

model = KeyedVectors.load_word2vec_format(
    os.path.join(model_path, 'glove-wiki-gigaword-50', 'glove-wiki-gigaword-50.gz'), binary=False)

In [9]:
leven_cluster = np.loadtxt("../aida-yago2-dataset/db_cluster_levestein_0_3.txt", dtype=np.int32)
lev_cluster_dict = {}
windows = 30
doc_windows = 20
for i, x in enumerate(leven_cluster):
    try:
        lev_cluster_dict[x].append(
            (mentions[i], entities[i], docs[i], word_indexes[i],
             ch.get_context(text_splitted[docs[i]], word_indexes[i], windows=windows)))
    except:
        lev_cluster_dict[x] = [
            (mentions[i], entities[i], docs[i], word_indexes[i],
             ch.get_context(text_splitted[docs[i]], word_indexes[i], windows=windows))]
# """se uso un cluster diverso da minimo 0 allora scommento"""
# del lev_cluster_dict[-1]

In [27]:
cluster_all = []
from sklearn.cluster import AgglomerativeClustering
from sklearn.cluster import DBSCAN

# from hdbscan import HDBSCAN

clusterizator = AgglomerativeClustering(n_clusters=None, affinity='cosine', distance_threshold=0.1, linkage="single")
# clusterizator = DBSCAN(min_samples=0, metric="cosine", n_jobs=-1, eps=0.1)
# clusterizator = HDBSCAN(min_cluster_size=2, metric="euclidean", min_samples=1, cluster_selection_epsilon=0.4)
vectors_all = []
for x in tqdm(lev_cluster_dict):
    contexts_vectorized = ch.calculate_context_vector(model, lev_cluster_dict[x], text_splitted, doc_window=doc_windows,
                                                      doc_context_alpha=0.3, word_context_aplha=.7)
    if len(contexts_vectorized) > 1:
        cluster = clusterizator.fit_predict(contexts_vectorized)
    else:
        cluster = np.zeros(1)
    cluster_all.append(cluster)
    vectors_all.append(contexts_vectorized)
    # print(cluster_all)
    # break

100%|██████████| 2750/2750 [00:04<00:00, 631.33it/s] 


In [32]:
lev_cluster_dict_vectorized = {}
for index in lev_cluster_dict.keys():
    lev_cluster_dict_vectorized[index] = []
    for i, element in enumerate(lev_cluster_dict[index]):
        # print((*element, vectors_all[index][i]))
        lev_cluster_dict_vectorized[index].append((*element, vectors_all[index][i]))

In [44]:
clustered_entities = [[(row[1], row[5]) for row in lev_cluster_dict_vectorized[index]] for index in lev_cluster_dict_vectorized.keys()]
len(clustered_entities)

2750

In [70]:
clusters_splitted = []
clusters_vectors = []
for n_cluster in range(len(clustered_entities)):
    cluster_unique = dict.fromkeys(set(cluster_all[n_cluster]), [])
    cluster_unique_vector = dict.fromkeys(set(cluster_all[n_cluster]), [])
    for key in cluster_unique:
        cluster_unique[key] = [clustered_entities[n_cluster][index][0] for index in
                               range(len(clustered_entities[n_cluster])) if cluster_all[n_cluster][index] == key]
        cluster_unique_vector[key] = [clustered_entities[n_cluster][index][1] for index in
                               range(len(clustered_entities[n_cluster])) if cluster_all[n_cluster][index] == key]
    clusters_vectors.append(cluster_unique_vector)
    clusters_splitted.append(cluster_unique)

In [112]:
clusters_splitted_vector_dict = [np.mean(x[y],axis=0)  for x in clusters_vectors for y in x if len(x[y]) > 1]

In [113]:
clusters_splitted_dict = [dict(Counter(x[y])) for x in clusters_splitted for y in x if len (x[y]) > 1]

In [118]:
clusterizator = AgglomerativeClustering(n_clusters=None, affinity='cosine', distance_threshold=0.001, linkage="single")

clusters_of_cluster = clusterizator.fit_predict(clusters_splitted_vector_dict)

In [119]:
for x in range(len(clusters_of_cluster)):
    print(clusters_of_cluster[x], clusters_splitted_dict[x])

1164 {'Germany': 351, 'German_language': 3, 'Germany_national_football_team': 6, 'Nazi_Germany': 2}
1651 {'United_Kingdom': 91, 'England': 1}
1070 {'Brussels': 42}
1411 {'Brussels': 2}
1417 {'European_Commission': 8}
1369 {'European_Union': 24}
2013 {'United_Kingdom': 165}
1337 {'European_Commission': 11}
1592 {'France': 193, 'France_national_football_team': 8}
1515 {'Bovine_spongiform_encephalopathy': 4}
1183 {'Budapest_Stock_Exchange': 2}
1744 {'Spain': 31}
2019 {'Europe': 44}
1707 {'Bonn': 42}
1727 {'Jimi_Hendrix': 4}
1179 {'London': 233, 'London_Wasps': 2, 'Harlequins_Rugby_League': 3, 'London_Stock_Exchange': 3}
1937 {'United_States': 467, 'United_Nations': 61, 'Hong_Kong': 4, 'United_Kingdom': 2, "United_States_men's_national_soccer_team": 2}
2075 {'England': 69, 'English_language': 6, 'United_Kingdom': 3}
1227 {'Nottingham': 2}
1244 {'Australia': 201, 'Australia_Davis_Cup_team': 4, 'Australia_national_cricket_team': 60, 'Australia_national_rugby_union_team': 7}
1119 {'Australia'

cosi facendo clusterizzo in categorie (insieme di entità) es. tutte le squadre di cricket sono clusterizzate insieme.